In [1]:
import numpy as np
from deck import full_euchre_deck
from dealer import dealer

In [2]:
game = dealer(deck=full_euchre_deck ,players=3)
game.deal_cards()
game.hand1, game.hand2, game.hand3

(array([[ 14,   0],
        [  0,  12],
        [  0,  10],
        [ 11,   0],
        [-10,   0]]),
 array([[  0, -14],
        [  0,  11],
        [-12,   0],
        [  0, -10],
        [  0,  13]]),
 array([[ 13,   0],
        [  0,   9],
        [ 12,   0],
        [-13,   0],
        [-14,   0]]))

In [3]:
game.hands

{'hand1': array([[ 14,   0],
        [  0,  12],
        [  0,  10],
        [ 11,   0],
        [-10,   0]]),
 'hand2': array([[  0, -14],
        [  0,  11],
        [-12,   0],
        [  0, -10],
        [  0,  13]]),
 'hand3': array([[ 13,   0],
        [  0,   9],
        [ 12,   0],
        [-13,   0],
        [-14,   0]])}

In [4]:
game.hands['hand4'] =  np.array([[-10,   0],
        [  0,   9],
        [  0,  11],
        [ -9,   0],
        [ 12,   0]])

In [5]:
game.hands

{'hand1': array([[ 14,   0],
        [  0,  12],
        [  0,  10],
        [ 11,   0],
        [-10,   0]]),
 'hand2': array([[  0, -14],
        [  0,  11],
        [-12,   0],
        [  0, -10],
        [  0,  13]]),
 'hand3': array([[ 13,   0],
        [  0,   9],
        [ 12,   0],
        [-13,   0],
        [-14,   0]]),
 'hand4': array([[-10,   0],
        [  0,   9],
        [  0,  11],
        [ -9,   0],
        [ 12,   0]])}

In [7]:
game.hand4 = game.hands['hand4']

In [8]:
game.hand4

array([[-10,   0],
       [  0,   9],
       [  0,  11],
       [ -9,   0],
       [ 12,   0]])